In [2]:
from os import listdir
from os.path import isdir
import xarray
from distributed import Client
import numpy as np
import dask


client = Client('localhost:8786')
client.restart()

<Client: 'tcp://146.6.192.149:8786' processes=10 threads=10>

In [ ]:
INPUT_DIR = "/projects/dgs/persad_research/Lochana_workspace/NEX-GDDP-CMIP6/"
OUTPUT_DIR  = "/projects/dgs/persad_research/SIMULATION_DATA/ZARR/NEX-GDDP-CMIP6/SIM_VARIABLES/"

models = [name for name in listdir(INPUT_DIR) if isdir(INPUT_DIR + name)]
calendars = {
    'NorESM2-MM': 'noleap',
    'CESM2': 'noleap',
    'BCC-CSM2-MR': 'noleap',
    'INM-CM4-8': 'noleap',
    'HadGEM3-GC31-LL': '360_day',
    'MPI-ESM1-2-LR': 'proleptic_gregorian',
    'NorESM2-LM': 'noleap',
    'FGOALS-g3': 'noleap',
    'NESM3': 'proleptic_gregorian',
    'IITM-ESM': 'proleptic_gregorian',
    'GFDL-ESM4': 'noleap',
    'GISS-E2-1-G': 'noleap',
    'ACCESS-CM2': 'proleptic_gregorian',
    'KIOST-ESM': 'noleap',
    'MIROC6': 'proleptic_gregorian',
    'MIROC-ES2L': 'proleptic_gregorian',
    'CMCC-CM2-SR5': 'noleap',
    'MRI-ESM2-0': 'proleptic_gregorian',
    'UKESM1-0-LL': '360_day',
    'IPSL-CM6A-LR': 'proleptic_gregorian',
    'GFDL-CM4': 'noleap',
    'GFDL-CM4_gr2': 'noleap',
    'KACE-1-0-G': '360_day',
    'CNRM-CM6-1': 'proleptic_gregorian',
    'EC-Earth3-Veg-LR': 'proleptic_gregorian',
    'CESM2-WACCM': 'noleap',
    'ACCESS-ESM1-5': 'proleptic_gregorian',
    'INM-CM5-0': 'noleap',
    'CanESM5': 'noleap',
    'HadGEM3-GC31-MM': '360_day',
    'CNRM-ESM2-1': 'proleptic_gregorian',
    'TaiESM1': 'noleap',
    'CMCC-ESM2': 'noleap',
    'EC-Earth3': 'proleptic_gregorian',
    'MPI-ESM1-2-HR': 'proleptic_gregorian'
}

dataset_groups = {}
for model in models:
    calendar = calendars[model]
    for exp in listdir(f"{INPUT_DIR}{model}/"):
        for em in listdir(f"{INPUT_DIR}{model}/{exp}/"):
            for variable in listdir(f"{INPUT_DIR}{model}/{exp}/{em}/"):
                dir_path = f"{INPUT_DIR}{model}/{exp}/{em}/{variable}/"
                zarr_name = f"{exp}_{calendar}_day_{variable}"
                model_exp = f"{model}"
                if zarr_name not in dataset_groups:
                    dataset_groups[zarr_name] = {}
                dataset_groups[zarr_name][model_exp] = [dir_path + name for name in listdir(dir_path)]

print("Building zarr repo...")
for index, group in enumerate(dataset_groups):
    datasets_merge = {}
    for model in dataset_groups[group]:
        variable = group.split("_")[-1]
        name = f"{model}_{variable}"
        ds = xarray.open_mfdataset(dataset_groups[group][model], chunks=dict(time=365, lat=-1, lon=-1)).chunk(dict(time=-1, lat=90, lon=72))
        da = ds[variable].rename(name)
        da.attrs |= ds.attrs
        datasets_merge[name] = da
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        merged_ds = xarray.Dataset(datasets_merge, coords=da.coords)
        merged_ds.attrs = {
            "contact": "cameron.cummins@utexas.edu",
            "description": "Large Zarr store for NEX-GDDP-CMIP6 multi-model ensembles. Not intended for distribution."
        }
        merged_ds.to_zarr(f"{OUTPUT_DIR}{group}.zarr", consolidated=True, mode="w")
    print(index, end=", ")

Building zarr repo...
